## Project Management Dashboard Data Generator

This script was intended to generate random data for my Project Management Overview Dashboard.

It creates a simulated dataset of employee work data for a given date range. 

#### Key features include:
 + Date Range: Business days between January 1, 2023, and August 1, 2023.
 + Employees and Projects: Selects from predefined lists with weighted probabilities.
 + Work Details:
     + Assignments: 1 or 2 projects per employee per day.
     + Hours Worked: Calculated within ±15% of the budgeted hours per project.
 + Revenue: Estimated based on hours worked.
 + Category: Randomly assigned as "Consulting", "Developing", or "Planning" with specific probabilities.
 + Output: Data saved as a CSV file, including columns for date, employee name, project details, hours worked, revenue, and category.

### Install Pandas

In [1]:
!pip install pandas

### Import Libraries

In [3]:
import pandas as pd
import random
import datetime

### Define Variables and Ranges

In [4]:
# Define the list of employees with weights
employees_with_weights = [
    ("Alex Johnson", 5),
    ("Emma Turner", 3),
    ("Michael Smith", 4),
    ("Sophia Brown", 2),
    ("James Wilson", 6),
    ("Olivia Garcia", 3),
    ("William Davis", 4),
    ("Isabella Martinez", 2)
]

# Define the projects with weights and run dates
projects_with_weights = [
    ("Tech Solutions Development", "Sarah Collins", 3, datetime.date(2023, 1, 1), datetime.date(2023, 6, 30)),
    ("Global Marketing Campaign", "David Kim", 3, datetime.date(2023, 2, 1), datetime.date(2023, 12, 31)),
    ("Product Innovation Lab", "Emily Patel", 1, datetime.date(2023, 1, 15), datetime.date(2023, 5, 31)),
    ("Enterprise Software Upgrade", "John Murphy", 2, datetime.date(2023, 3, 1), datetime.date(2023, 11, 30)),
    ("Strategic Client Acquisition", "Angela Zhao", 4, datetime.date(2023, 4, 1), datetime.date(2023, 10, 31))
]

# Define start and end dates for data
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 8, 1)

### Create Work Data Generator Function

In [6]:
def generate_work_data_for_all(start_date, end_date):
    all_days = pd.date_range(start_date, end_date, freq='B').date
    work_data = []
    
    # Choose all days in date range
    for day in all_days:
        # Randomly choose an employee
        employee_info = random.choice(employees_with_weights)
        employee_name = employee_info[0]

        # Randomly decide if the employee is not scheduled (5% chance)
        if random.random() < 0.05:
            continue

        # Deciding on the number of projects for the day
        num_projects = random.randint(1, 2)
        total_hours_available = 8

        for _ in range(num_projects):
            # Filtering valid projects for the day
            # A project is considered valid for thr day if that day falls within the project's start and end dates
            valid_projects = []
            for project in projects_with_weights:
                if project[3] <= day <= project[4]:
                    valid_projects.append(project)

            if not valid_projects:
                continue

            # Choose a project randomly
            project_info = random.choice(valid_projects)
            project_name = project_info[0]
            project_manager = project_info[1]

            # Calculate budgeted hours per project
            budgeted_hours_per_project = total_hours_available / num_projects

            # Calulcate hours worked with a +/-15% deviation
            deviation = budgeted_hours_per_project * 0.15
            min_hours = max(0, budgeted_hours_per_project - deviation)
            max_hours = min(total_hours_available, budgeted_hours_per_project + deviation)
            hours_worked = round(random.uniform(min_hours, max_hours), 1)

            # Use above to reduce total hours available for next project
            total_hours_available -= hours_worked

            # Calulcate revenue
            revenue = round(hours_worked * random.uniform(100, 500), 2)

            # Choose a category for the work
            categories = ["Consulting", "Developing", "Planning"]
            weights = [8, 5, 1]
            category = random.choices(categories, weights)[0]

            # Appending the work data to the list
            work_data.append({
                'Date': day,
                'Employee Name': employee_name,
                'Project': project_name,
                'Project Manager': project_manager,
                'Hours Worked': hours_worked,
                'Budgeted Hours': budgeted_hours_per_project,
                'Revenue': revenue,
                'Category': category
            })

    # Convert list to DataFrame
    return pd.DataFrame(work_data)

### Generate the DataFrame and Save to CSV

In [13]:
df = generate_work_data_for_all(start_date, end_date)

file_name = 'employee_work_data.csv'
df.to_csv(file_name, index=False)

print(f"Employee work data has been saved to {file_name}")

Employee work data has been saved to employee_work_data.csv
